In [150]:
pip install ortools

# Class Exercise

In [178]:
import pandas as pd
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp


In [152]:
class data_generator():

  def __init__(self):
    self.setup_cost = 5000
    self.production_cost = 100
    self.first_inventory = 200
    self.inventory_cost = 5

    self.months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug']
    self.demands = [400, 400, 800, 800, 1200, 1200, 1200, 1200]

In [186]:
# Optimization Function
def opt_production_first(data):
    m = pywraplp.Solver.CreateSolver('SCIP')

    # Decision variables
    setup = []  # Binary variable (1 if production occurs, 0 otherwise)
    production = []  # Quantity produced each month
    inventory = []  # Inventory at the end of each month

    for month in range(len(data.months)):
        suffix = '_{}'.format(month)
        setup.append(m.BoolVar('setup' + suffix))
        production.append(m.NumVar(0, sum(data.demands), 'production' + suffix))
        inventory.append(m.NumVar(0, sum(data.demands), 'inventory' + suffix))

    # Constraints
    # Constraints
    # Flow conservation for the first month
    m.Add(data.first_inventory + production[0] == data.demands[0] + inventory[0])

    # 🔹 **Force setup = 1 if there is production in the first month**
    m.Add(setup[0] >= production[0] / sum(data.demands))  # Ensures setup is 1 if any production occurs

    for month in range(1, len(data.months)):
        # If production happens, setup must be 1
        m.Add(production[month] <= sum(data.demands) * setup[month])
        m.Add(production[month] >= 0.1 * setup[month])

        # Inventory balance constraint
        m.Add(inventory[month-1] + production[month] == data.demands[month] + inventory[month])

    # Objective function: Minimize total costs (setup, production, and inventory)
    obj = m.Minimize(
        sum(setup[m_idx] * data.setup_cost for m_idx in range(len(data.months))) +
        sum(production[m_idx] * data.production_cost for m_idx in range(len(data.months))) +
        sum(inventory[m_idx] * data.inventory_cost for m_idx in range(len(data.months)-1)) +
        inventory[-1] * data.inventory_cost / 2  # End inventory cost
    )

    # Solve the model
    status = m.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        PRODUCTION_PLAN = {}
        for m_idx, month in enumerate(data.months):
            PRODUCTION_PLAN[month] = {
                'production': production[m_idx].solution_value(),
                'setup': round(setup[m_idx].solution_value()),
                'inventory': inventory[m_idx].solution_value()
            }
        return PRODUCTION_PLAN
    else:
        print('The problem does not have an optimal solution.')
        return None

# Execute the model and calculate costs
data = data_generator()
PRODUCTION_PLAN = opt_production_first(data)

if PRODUCTION_PLAN:
    # Cost computation dictionary
    costs = {
        "Setup Cost": [],
        "Production Cost": [],
        "Inventory Cost": [],
        "Total Cost": []
    }

    total_setup_cost = 0
    total_production_cost = 0
    total_inventory_cost = 0

    for m_idx, month in enumerate(data.months):
        setup_value = PRODUCTION_PLAN[month]['setup']
        production_value = PRODUCTION_PLAN[month]['production']
        inventory_value = PRODUCTION_PLAN[month]['inventory']

        # Use the solver's setup variable directly
        setup_cost = setup_value * data.setup_cost
        production_cost = production_value * data.production_cost
        inventory_cost = inventory_value * data.inventory_cost

        # Append costs to the dictionary
        costs["Setup Cost"].append(setup_cost)
        costs["Production Cost"].append(production_cost)
        costs["Inventory Cost"].append(inventory_cost)
        costs["Total Cost"].append(setup_cost + production_cost + inventory_cost)

        # Accumulate total costs
        total_setup_cost += setup_cost
        total_production_cost += production_cost
        total_inventory_cost += inventory_cost

    # Create a DataFrame for costs
    costs_df = pd.DataFrame(costs, index=data.months)

    # Format numerical values to two decimal places with thousands separators
    costs_df_formatted = costs_df.applymap(lambda x: f"{x:,.2f}")

    # Display the formatted DataFrame
    from IPython.display import display
    display(costs_df_formatted)

    # Display total cost breakdown
    total_cost = total_setup_cost + total_production_cost + total_inventory_cost
    print(f" Total Plan Cost: {total_cost:,.2f}")
    print(f"   - Setup Cost: {total_setup_cost:,.2f}")
    print(f"   - Production Cost: {total_production_cost:,.2f}")
    print(f"   - Inventory Cost: {total_inventory_cost:,.2f}")

else:
    print("No data available to display.")


<ipython-input-186-ba687a76be9d>:98: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  costs_df_formatted = costs_df.applymap(lambda x: f"{x:,.2f}")


,Setup Cost,Production Cost,Inventory Cost,Total Cost
Jan,"5,000.00","60,000.00","2,000.00","67,000.00"
Feb,0.00,0.00,0.00,0.00
Mar,"5,000.00","160,000.00","4,000.00","169,000.00"
Apr,0.00,0.00,-0.00,-0.00
May,"5,000.00","120,000.00",0.00,"125,000.00"
Jun,"5,000.00","120,000.00",0.00,"125,000.00"
Jul,"5,000.00","120,000.00",-0.00,"125,000.00"
Aug,"5,000.00","120,000.00",0.00,"125,000.00"


 Total Plan Cost: 736,000.00
   - Setup Cost: 30,000.00
   - Production Cost: 700,000.00
   - Inventory Cost: 6,000.00


In [187]:
data = data_generator()
PRODUCTION_PLAN = opt_production_first(data)
display(pd.DataFrame.from_dict(PRODUCTION_PLAN).round(0))



,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug
production,600.0,0.0,1600.0,0.0,1200.0,1200.0,1200.0,1200.0
setup,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
inventory,400.0,0.0,800.0,-0.0,0.0,0.0,-0.0,0.0


In [188]:
# Ejecutar modelo
data = data_generator()
PRODUCTION_PLAN = opt_production_first(data)

# 🔹 Imprimir valores antes de mostrar la tabla
print("\n📊 **Resultados de la Optimización:**")
print("Mes\tProducción\tSetup\tInventario")
for month in data.months:
    print(f"{month}\t{PRODUCTION_PLAN[month]['production']:.0f}\t\t{PRODUCTION_PLAN[month]['setup']}\t\t{PRODUCTION_PLAN[month]['inventory']:.0f}")

# 🔹 Mostrar tabla corregida
display(pd.DataFrame.from_dict(PRODUCTION_PLAN).round(0))



📊 **Resultados de la Optimización:**
Mes	Producción	Setup	Inventario
Jan	600		1		400
Feb	0		0		0
Mar	1600		1		800
Apr	0		0		-0
May	1200		1		0
Jun	1200		1		0
Jul	1200		1		-0
Aug	1200		1		0


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug
production,600.0,0.0,1600.0,0.0,1200.0,1200.0,1200.0,1200.0
setup,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
inventory,400.0,0.0,800.0,-0.0,0.0,0.0,-0.0,0.0


## Question 1

In [189]:
class data_generator_second():

  def __init__(self):
    self.setup_cost = 3000
    self.production_cost = 60
    self.first_inventory = 0
    self.inventory_cost = 3

    self.months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug']
    self.demands = [200, 200, 200, 200, 200, 200, 500, 500]

In [190]:
data = data_generator_second()
PRODUCTION_PLAN = opt_production_first(data)
display(pd.DataFrame.from_dict(PRODUCTION_PLAN).round(0))

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug
production,600.0,0.0,0.0,600.0,-0.0,-0.0,1000.0,0.0
setup,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
inventory,400.0,200.0,0.0,400.0,200.0,-0.0,500.0,0.0


## Question 2

In [194]:
class data_generator_third():

  def __init__(self):
    self.setup_cost = [5000, 3000]
    self.production_cost = [100, 60]
    self.first_inventory = [200,0]
    self.inventory_cost = [5, 3]

    self.months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug']
    self.demands = [[400, 400, 800, 800, 1200, 1200, 1200, 1200], \
                    [200, 200, 200, 200, 200, 200, 500, 500]]

In [195]:
from ortools.linear_solver import pywraplp
import pandas as pd

def opt_production_second(data):

    # Create the mip solver with the SCIP backend.
    m = pywraplp.Solver.CreateSolver('SCIP')

    # Decision variables
    obj = m.NumVar(0, 1000000, 'obj')

    setup = []
    production = []
    inventory = []
    for bike in range(len(data.demands)):
        setup_temp = []
        production_temp = []
        inventory_temp = []
        for month in range(len(data.months)):
            suffix = '_{}_{}'.format(bike, month)
            setup_temp.append(m.BoolVar('setup' + suffix))
            production_temp.append(m.NumVar(0, sum(data.demands[bike]), 'production' + suffix))
            inventory_temp.append(m.NumVar(0, sum(data.demands[bike]), 'inventory' + suffix))

        setup.append(setup_temp)
        production.append(production_temp)
        inventory.append(inventory_temp)

    # Constraints
    for bike in range(len(data.demands)):
        # **First-Month Inventory Balance Constraint**
        m.Add(data.first_inventory[bike] + production[bike][0] == data.demands[bike][0] + inventory[bike][0])

        # 🔹 **Force setup = 1 in the first month if production occurs**
        m.Add(setup[bike][0] >= production[bike][0] / sum(data.demands[bike]))

        for month in range(1, len(data.months)):
            # **Setup constraint**
            m.Add(production[bike][month] <= sum(data.demands[bike][month:]) * setup[bike][month])

            # **Flow conservation constraint**
            m.Add(inventory[bike][month-1] + production[bike][month] == data.demands[bike][month] + inventory[bike][month])

    # Objective function constraint
    m.Add(obj >= sum(
        sum(setup[bike]) * data.setup_cost[bike] + \
        sum(production[bike]) * data.production_cost[bike] + \
        sum(inventory[bike][:-1]) * data.inventory_cost[bike] + \
        inventory[bike][-1] * data.inventory_cost[bike] / 2
        for bike in range(len(data.demands))
    ))

    # Minimize objective
    m.Minimize(obj)

    # Solve the model
    status = m.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        PRODUCTION_PLAN = {}
        for bike in range(len(data.demands)):
            PRODUCTION_PLAN_temp = {}
            for m_idx, month in enumerate(data.months):
                PRODUCTION_PLAN_temp[month] = {
                    'production': production[bike][m_idx].solution_value(),
                    'setup': round(setup[bike][m_idx].solution_value()),  # Round setup to ensure it's 0 or 1
                    'inventory': inventory[bike][m_idx].solution_value()
                }
            PRODUCTION_PLAN[bike] = PRODUCTION_PLAN_temp
        return PRODUCTION_PLAN, obj.solution_value()

    else:
        print('The problem does not have an optimal solution.')
        return None, None

# Run the function
data = data_generator_third()
PRODUCTION_PLAN, obj = opt_production_second(data)

if PRODUCTION_PLAN:
    display(pd.DataFrame.from_dict(PRODUCTION_PLAN[0]).round(2))
    display(pd.DataFrame.from_dict(PRODUCTION_PLAN[1]).round(2))
    print('Objective is:', round(obj, 2))


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug
production,600.0,0.0,1600.0,0.0,1200.0,1200.0,1200.0,1200.0
setup,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
inventory,400.0,0.0,800.0,0.0,0.0,0.0,0.0,0.0


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug
production,600.0,0.0,0.0,600.0,0.0,0.0,1000.0,0.0
setup,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
inventory,400.0,200.0,0.0,400.0,200.0,0.0,500.0,0.0


Objective is: 882100.0


## Question 3

In [198]:
from ortools.linear_solver import pywraplp
import pandas as pd

def opt_production_third(data):

    # Create the mip solver with the SCIP backend.
    m = pywraplp.Solver.CreateSolver('SCIP')

    # Decision variables
    obj = m.NumVar(0, 1000000, 'obj')

    setup = []
    production = []
    inventory = []
    for bike in range(len(data.demands)):
        setup_temp = []
        production_temp = []
        inventory_temp = []
        for month in range(len(data.months)):
            suffix = '_{}_{}'.format(bike, month)
            setup_temp.append(m.BoolVar('setup' + suffix))
            production_temp.append(m.NumVar(0, sum(data.demands[bike]), 'production' + suffix))
            inventory_temp.append(m.NumVar(0, sum(data.demands[bike]), 'inventory' + suffix))

        setup.append(setup_temp)
        production.append(production_temp)
        inventory.append(inventory_temp)

    # Objective function constraint
    m.Add(obj >= sum(
        sum(setup[bike]) * data.setup_cost[bike] +
        sum(production[bike]) * data.production_cost[bike] +
        sum(inventory[bike][:-1]) * data.inventory_cost[bike] +
        inventory[bike][-1] * data.inventory_cost[bike] / 2
        for bike in range(len(data.demands))
    ))

    # Constraints
    for bike in range(len(data.demands)):
        # **First-Month Inventory Balance Constraint**
        m.Add(data.first_inventory[bike] + production[bike][0] == data.demands[bike][0] + inventory[bike][0])

        # 🔹 **Force setup = 1 in the first month if production occurs**
        m.Add(setup[bike][0] >= production[bike][0] / sum(data.demands[bike]))

        for month in range(1, len(data.months)):
            # **Setup constraint**
            m.Add(production[bike][month] <= sum(data.demands[bike][month:]) * setup[bike][month])

            # **Flow conservation constraint**
            m.Add(inventory[bike][month-1] + production[bike][month] == data.demands[bike][month] + inventory[bike][month])

    # **Total production limit per month**
    for month in range(1, len(data.months)):
        m.Add(sum(production[bike][month] for bike in range(len(data.demands))) <= 1500)

    # Minimize objective
    m.Minimize(obj)

    # Solve the model
    status = m.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        PRODUCTION_PLAN = {}
        for bike in range(len(data.demands)):
            PRODUCTION_PLAN_temp = {}
            for m_idx, month in enumerate(data.months):
                PRODUCTION_PLAN_temp[month] = {
                    'production': production[bike][m_idx].solution_value(),
                    'setup': round(setup[bike][m_idx].solution_value()),  # Round setup to ensure it's 0 or 1
                    'inventory': inventory[bike][m_idx].solution_value()
                }
            PRODUCTION_PLAN[bike] = PRODUCTION_PLAN_temp
        return PRODUCTION_PLAN, obj.solution_value()

    else:
        print('The problem does not have an optimal solution.')
        return None, None

# Run the function
data = data_generator_third()
PRODUCTION_PLAN, obj = opt_production_third(data)

if PRODUCTION_PLAN:
    display(pd.DataFrame.from_dict(PRODUCTION_PLAN[0]).round(2))
    display(pd.DataFrame.from_dict(PRODUCTION_PLAN[1]).round(2))
    print('Objective is:', round(obj, 2))


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug
production,700.0,-0.0,1500.0,0.0,1200.0,1200.0,1400.0,1000.0
setup,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
inventory,500.0,100.0,800.0,0.0,0.0,0.0,200.0,0.0


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug
production,600.0,0.0,0.0,1100.0,0.0,0.0,0.0,500.0
setup,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
inventory,400.0,200.0,0.0,900.0,700.0,500.0,0.0,0.0


Objective is: 887100.0
